# 09 ago 2018

- [ ]  Migrar todos os dados relativos a proposições coletados anteriormente
- [ ]  Estudar `views` do sql para aumentar a velocidade de acesso aos dados
- [ ]  Estudar maneiras de obter dados do judiciário

In [1]:
import pandas as pd
import numpy as np
import requests
import sys
from utils.camara_obter_dados import obter_dados
from utils.dataframe_to_file import dataframe_to_file
from utils.dataframe_to_database import dataframe_to_database

## Importando Datasets

### Proposições enviadas pelos Deputados

Dimensões do Dataset gerado: `375814 rows x 7 columns`

In [18]:
#     '../data/2018-07-30-camara-deputados-proposicoes.xz',
#     compression='xz',

In [19]:
df_proposicoes_deputados = pd.read_csv(
    '../data/2018-07-30-camara-deputados-proposicoes',
)

df_proposicoes_deputados

,idDeputado,idProposicao,siglaTipo,idTipo,numero,ano,ementa
0,3151,354049,PRL,190,1,0,"Parecer do Relator, Dep. Jairo Ataide, pela ap..."
1,3151,354055,EMR,257,1,0,"Altera o inciso IV do art. 5º da Lei nº 7.827,..."
2,3151,359251,PRL,190,2,0,"Parecer do Relator, Dep. Jairo Ataide (DEM-MG)..."
3,3151,363698,PRL,190,1,0,"Parecer do Relator, Dep. Jairo Ataide (DEM-MG)..."
4,3151,371165,VTS,260,1,0,"Modifica o art. 2ª da Lei Complementar nº 125,..."
5,3151,396123,PRL,190,1,0,"Parecer do Relator, Dep. Jairo Ataide (DEM-MG)..."
6,3151,396151,EMR,257,1,0,"Acrescenta artigo 7º ao PL, renumerando-se os ..."
7,3151,427862,PRL,190,1,0,"Parecer do Relator, Dep. Jairo Ataide (DEM-MG)..."
8,3151,433731,PRL,190,1,0,"Parecer do Relator, Dep. Jairo Ataide (DEM-MG)..."
9,3151,435523,PRL,190,2,0,"Parecer do Relator, Dep. Jairo Ataide (DEM-MG)..."


Dimensões do Dataset Importado: `375814 rows x 7 columns`

_Obs.: Foi preciso deletar a quebra de linha entre as linhas 371422 e 371423 do dataset para importar corretamente_

### Proposições enviadas pelo Poder Executivo

Dimensões do Dataset gerado: `24813 rows x 7 columns`

In [20]:
df_proposicoes_executivo = pd.read_csv(
    '../data/2018-07-31-camara-proposicoes-poder-executivo'
)

df_proposicoes_executivo

,ano,ementa,id,idTipo,numero,siglaTipo,uri
0,0,Submete à consideração do Congresso Nacional o...,273785,187,1,PAR,https://dadosabertos.camara.leg.br/api/v2/prop...
1,0,Dispõe sobre a gestão de florestas públicas pa...,288300,701,1,SBT-A,https://dadosabertos.camara.leg.br/api/v2/prop...
2,0,Parecer de Comissão.,288568,187,1,PAR,https://dadosabertos.camara.leg.br/api/v2/prop...
3,0,Aprovado por Unanimidade o Parecer com Complem...,349546,187,1,PAR,https://dadosabertos.camara.leg.br/api/v2/prop...
4,0,"Altera dispositivos da Lei nº 8.666, de 21 de ...",349549,701,1,SBT-A,https://dadosabertos.camara.leg.br/api/v2/prop...
5,0,Aprovado o Parecer com Complementação de Voto ...,349585,187,1,PAR,https://dadosabertos.camara.leg.br/api/v2/prop...
6,0,"Aprovado o Parecer.. Parecer do Relator, Dep....",429092,187,1,PAR,https://dadosabertos.camara.leg.br/api/v2/prop...
7,0,"Aprovado o Parecer, por unanimidade, com subst...",432091,187,1,PAR,https://dadosabertos.camara.leg.br/api/v2/prop...
8,0,Aprovado por Unanimidade o Parecer.. Parecer ...,432652,187,1,PAR,https://dadosabertos.camara.leg.br/api/v2/prop...
9,0,SUBSTITUTIVO ADOTADO PELA COMISSÃO,459557,701,1,SBT-A,https://dadosabertos.camara.leg.br/api/v2/prop...


Dimensões do Dataset Importado: `24813 rows x 7 columns`

### Proposições de Destaque (PEC, PL, PLP, PFC, PDC)

Dimensões do Dataset gerado: `38333 rows x 32 columns`

In [24]:
df_proposicoes_detalhadas = pd.read_csv(
    '../data/2018-08-07-camara-proposicoes-detalhadas',
    dtype={
        'id': np.int32,
        'uri': str,
        'siglaTipo': str,
        'idTipo': str,
        'numero': np.int32,
        'ano': np.int32,
        'ementa': str
    }
)

df_proposicoes_detalhadas

,id,uri,siglaTipo,idTipo,numero,ano,ementa,dataApresentacao,uriOrgaoNumerador,uriUltimoRelator,...,descricaoTipo,ementaDetalhada,keywords,uriPropPrincipal,uriPropAnterior,uriPropPosterior,urlInteiroTeor,urnFinal,texto,justificativa
0,103385,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,7511,2003,"Altera dispositivos da Lei nº 7.501, de 27 de ...",2003-01-03T15:54,NaN,NaN,...,Projeto de Lei,NaN,"Alteração, Regime Jurídico, servidor, Serviço ...",NaN,NaN,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,NaN,NaN
1,103442,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,7512,2003,"Prorroga a vigência da Lei nº 8.989, de 24 de ...",2003-01-02T18:11,NaN,NaN,...,Projeto de Lei,NaN,"Alteração, prorrogação, prazo, vigência, Lei d...",NaN,NaN,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,NaN,NaN
2,103648,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,7514,2003,"Altera a Lei nº 7.070, de 20 de dezembro de 19...",2003-01-14T15:46,NaN,NaN,...,Projeto de Lei,NaN,"Alteração, lei federal, concessão, pensão espe...",NaN,NaN,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,NaN,NaN
3,104314,https://dadosabertos.camara.leg.br/api/v2/prop...,PEC,136,1,2003,Altera a redação do art. 55 da Constituição F...,2003-02-18T14:27,NaN,NaN,...,Proposta de Emenda à Constituição,NaN,"Alteração, Constituição Federal, Legislativo, ...",NaN,NaN,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,NaN,NaN
4,104324,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,2,2003,"Altera o Decreto - Lei nº 2.848, de 7 de dezem...",2003-02-18T14:31,NaN,NaN,...,Projeto de Lei,NaN,"Alteração, legislação trabalhista, (CLT), proi...",https://dadosabertos.camara.leg.br/api/v2/prop...,NaN,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,NaN,NaN
5,104325,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,3,2003,"Altera o art. 129, do Decreto - Lei nº 2.848, ...",2003-02-18T14:31,NaN,NaN,...,Projeto de Lei,NaN,"Alteração, Código Penal, caracterização, crime...",NaN,NaN,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,NaN,NaN
6,104326,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,4,2003,Proíbe a participação de agentes públicos poli...,2003-02-18T14:32,NaN,NaN,...,Projeto de Lei,NaN,"Proibição, participação, policial, Polícia Fed...",NaN,NaN,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,NaN,NaN
7,104327,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,5,2003,"Altera os arts. 1º e 20 da Lei nº 7.716, de 5 ...",2003-02-18T14:32,NaN,NaN,...,Projeto de Lei,NaN,"_ Alteração, Lei Anti-Racismo, penalidade, inf...",NaN,NaN,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,NaN,NaN
8,104328,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,6,2003,"Altera a Lei nº 8.112, de 11 de dezembro de 19...",2003-02-18T14:33,NaN,NaN,...,Projeto de Lei,NaN,"Alteração, Regime Jurídico Único, proibição, s...",NaN,NaN,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,NaN,NaN
9,104329,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,7,2003,Dispõe sobre a criação de Programa de Orientaç...,2003-02-18T14:33,NaN,NaN,...,Projeto de Lei,NaN,"Criação, Programa de Orientação Sexual, de Pre...",NaN,NaN,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,NaN,NaN


Dimensões do Dataset Importado: `38333 rows x 32 columns`

_Obs.: Foi preciso deletar a quebra de linha entre as linhas 30400 e 30401 do dataset para importar corretamente_

## Migrar datasets para o banco de dados

### camara_proposicao - legislativo

In [39]:
df_proposicoes_camara_unicas = (df_proposicoes_deputados
    .drop_duplicates('idProposicao', keep='last')
    .sort_values(by=['idProposicao'], ascending=True, na_position='first')
)
df_proposicoes_camara_unicas.index = df_proposicoes_camara_unicas.reset_index().index

df_proposicoes_camara_unicas['ementa'] = df_proposicoes_camara_unicas.ementa.fillna('')

df_proposicoes_camara_unicas

,idDeputado,idProposicao,siglaTipo,idTipo,numero,ano,ementa
0,74200,12666,CON,129,3,1999,Solicita o pronunciamento da Comissão de Const...
1,74520,12668,CON,129,4,1995,SOLICITA O PRONUNCIAMENTO DA COMISSÃO DE CONST...
2,74218,12669,CON,129,4,2000,Consulta acerca da possibilidade de assunção t...
3,74099,12678,INC,133,38,1999,"SUGERE AO PODER EXECUTIVO, POR INTERMEDIO DO M..."
4,74779,12682,INC,133,50,1999,"SUGERE AO PODER EXECUTIVO, POR INTERMEDIO DO M..."
5,74099,12685,INC,133,68,1999,"SUGERE AO PODER EXECUTIVO, POR INTERMEDIO DO M..."
6,74779,12689,INC,133,103,1999,"SUGERE AO PODER EXECUTIVO, POR INTERMEDIO DA ..."
7,74479,12695,INC,133,124,1999,"SUGERE AO PODER EXECUTIVO, POR INTERMEDIO DO M..."
8,74447,12696,INC,133,126,1999,"SUGERE AO PODER EXECUTIVO, POR INTERMÉDIO DO M..."
9,74790,12697,INC,133,140,1999,"SUGERE A PODER EXECUTIVO, POR INTERMEDIO DO MI..."


In [40]:
df_proposicoes_deputados_limpa = df_proposicoes_deputados.ementa.fillna('')

dataframe_to_database(
    dataframe=df_proposicoes_camara_unicas,
    tabela='camara_proposicao',
    colunas_tabela=['id', 'tipo_id', 'tipo_sigla', 'numero', 'ano', 'ementa'],
    colunas_dataframe=['idProposicao', 'idTipo', 'siglaTipo', 'numero', 'ano', 'ementa']
)

373704 de 373704

### camara_proposicoes - executivo

In [46]:
df_proposicoes_executivo.drop_duplicates('id', keep='last')

,ano,ementa,id,idTipo,numero,siglaTipo,uri
0,0,Submete à consideração do Congresso Nacional o...,273785,187,1,PAR,https://dadosabertos.camara.leg.br/api/v2/prop...
1,0,Dispõe sobre a gestão de florestas públicas pa...,288300,701,1,SBT-A,https://dadosabertos.camara.leg.br/api/v2/prop...
2,0,Parecer de Comissão.,288568,187,1,PAR,https://dadosabertos.camara.leg.br/api/v2/prop...
3,0,Aprovado por Unanimidade o Parecer com Complem...,349546,187,1,PAR,https://dadosabertos.camara.leg.br/api/v2/prop...
4,0,"Altera dispositivos da Lei nº 8.666, de 21 de ...",349549,701,1,SBT-A,https://dadosabertos.camara.leg.br/api/v2/prop...
5,0,Aprovado o Parecer com Complementação de Voto ...,349585,187,1,PAR,https://dadosabertos.camara.leg.br/api/v2/prop...
6,0,"Aprovado o Parecer.. Parecer do Relator, Dep....",429092,187,1,PAR,https://dadosabertos.camara.leg.br/api/v2/prop...
7,0,"Aprovado o Parecer, por unanimidade, com subst...",432091,187,1,PAR,https://dadosabertos.camara.leg.br/api/v2/prop...
8,0,Aprovado por Unanimidade o Parecer.. Parecer ...,432652,187,1,PAR,https://dadosabertos.camara.leg.br/api/v2/prop...
9,0,SUBSTITUTIVO ADOTADO PELA COMISSÃO,459557,701,1,SBT-A,https://dadosabertos.camara.leg.br/api/v2/prop...


In [44]:
df_proposicoes_executivo_unicas = (df_proposicoes_executivo
    .drop_duplicates('id', keep='last')
    .sort_values(by=['id'], ascending=True, na_position='first')
)
df_proposicoes_executivo_unicas.index = df_proposicoes_executivo_unicas.reset_index().index

df_proposicoes_executivo_unicas['ementa'] = df_proposicoes_executivo_unicas.ementa.fillna('')

df_proposicoes_executivo_unicas

,ano,ementa,id,idTipo,numero,siglaTipo,uri
0,1997,Submete à apreciação do Congresso Nacional o a...,13032,134,3,MSC,https://dadosabertos.camara.leg.br/api/v2/prop...
1,2000,Encaminha ao Congresso Nacional a retificação ...,13033,134,3,MSC,https://dadosabertos.camara.leg.br/api/v2/prop...
2,1997,Submete à apreciação do Congresso Nacional o a...,13034,134,5,MSC,https://dadosabertos.camara.leg.br/api/v2/prop...
3,2001,Submete à consideração do Congresso Nacional o...,13035,411,6,MSC,https://dadosabertos.camara.leg.br/api/v2/prop...
4,1997,Submete à apreciação do Congresso Nacional o a...,13036,134,14,MSC,https://dadosabertos.camara.leg.br/api/v2/prop...
5,1995,Submete à consideração do Congresso Nacional o...,13038,134,24,MSC,https://dadosabertos.camara.leg.br/api/v2/prop...
6,1999,Submete à apreciação do Congresso Nacional o a...,13039,134,25,MSC,https://dadosabertos.camara.leg.br/api/v2/prop...
7,1995,Submete à consideração do Congresso Nacional o...,13040,134,29,MSC,https://dadosabertos.camara.leg.br/api/v2/prop...
8,1995,Submete à consideração do Congresso Nacional o...,13041,134,32,MSC,https://dadosabertos.camara.leg.br/api/v2/prop...
9,1998,Submete à apreciação do Congresso Nacional o a...,13042,134,34,MSC,https://dadosabertos.camara.leg.br/api/v2/prop...


In [48]:
dataframe_to_database(
    dataframe=df_proposicoes_executivo_unicas,
    tabela='camara_proposicao',
    colunas_tabela=['id', 'tipo_id', 'tipo_sigla', 'numero', 'ano', 'ementa'],
    colunas_dataframe=['id', 'idTipo', 'siglaTipo', 'numero', 'ano', 'ementa']
)

24680 de 24680

### camara_proposicao_deputado

In [49]:
df_proposicoes_deputados

,idDeputado,idProposicao,siglaTipo,idTipo,numero,ano,ementa
0,3151,354049,PRL,190,1,0,"Parecer do Relator, Dep. Jairo Ataide, pela ap..."
1,3151,354055,EMR,257,1,0,"Altera o inciso IV do art. 5º da Lei nº 7.827,..."
2,3151,359251,PRL,190,2,0,"Parecer do Relator, Dep. Jairo Ataide (DEM-MG)..."
3,3151,363698,PRL,190,1,0,"Parecer do Relator, Dep. Jairo Ataide (DEM-MG)..."
4,3151,371165,VTS,260,1,0,"Modifica o art. 2ª da Lei Complementar nº 125,..."
5,3151,396123,PRL,190,1,0,"Parecer do Relator, Dep. Jairo Ataide (DEM-MG)..."
6,3151,396151,EMR,257,1,0,"Acrescenta artigo 7º ao PL, renumerando-se os ..."
7,3151,427862,PRL,190,1,0,"Parecer do Relator, Dep. Jairo Ataide (DEM-MG)..."
8,3151,433731,PRL,190,1,0,"Parecer do Relator, Dep. Jairo Ataide (DEM-MG)..."
9,3151,435523,PRL,190,2,0,"Parecer do Relator, Dep. Jairo Ataide (DEM-MG)..."


In [50]:
dataframe_to_database(
    dataframe=df_proposicoes_deputados,
    tabela='camara_proposicao_deputado',
    colunas_tabela=['camara_proposicao_id', 'deputado_id'],
    colunas_dataframe=['idProposicao', 'idDeputado']
)

375814 de 375814

### camara_proposicao_detalhada

In [70]:
df_proposicoes_detalhadas[[
    'id',
    'dataApresentacao',
    'statusProposicao_dataHora',
    'statusProposicao_descricaoTramitacao',
    'statusProposicao_descricaoSituacao',
    'keywords',
    'urlInteiroTeor'
]]

,id,dataApresentacao,statusProposicao_dataHora,statusProposicao_descricaoTramitacao,statusProposicao_descricaoSituacao,keywords,urlInteiroTeor
0,103385,2003-01-03T15:54,2004-11-16T00:00,Arquivamento,Transformado em Norma Jurídica,"Alteração, Regime Jurídico, servidor, Serviço ...",http://www.camara.gov.br/proposicoesWeb/prop_m...
1,103442,2003-01-02T18:11,2005-06-08T00:00,Publicação de Parecer,Arquivada,"Alteração, prorrogação, prazo, vigência, Lei d...",http://www.camara.gov.br/proposicoesWeb/prop_m...
2,103648,2003-01-14T15:46,2004-06-04T00:00,Transformação em Norma Jurídica,Transformado em Norma Jurídica,"Alteração, lei federal, concessão, pensão espe...",http://www.camara.gov.br/proposicoesWeb/prop_m...
3,104314,2003-02-18T14:27,2012-05-18T00:00,Notificações,Arquivada,"Alteração, Constituição Federal, Legislativo, ...",http://www.camara.gov.br/proposicoesWeb/prop_m...
4,104324,2003-02-18T14:31,2007-01-31T00:00,Arquivamento,Arquivada,"Alteração, legislação trabalhista, (CLT), proi...",http://www.camara.gov.br/proposicoesWeb/prop_m...
5,104325,2003-02-18T14:31,2004-06-17T00:00,Transformação em Norma Jurídica,Transformado em Norma Jurídica,"Alteração, Código Penal, caracterização, crime...",http://www.camara.gov.br/proposicoesWeb/prop_m...
6,104326,2003-02-18T14:32,2008-03-06T00:00,Notificações,Arquivada,"Proibição, participação, policial, Polícia Fed...",http://www.camara.gov.br/proposicoesWeb/prop_m...
7,104327,2003-02-18T14:32,2007-03-23T00:00,Desarquivamento a Pedido,Arquivada,"_ Alteração, Lei Anti-Racismo, penalidade, inf...",http://www.camara.gov.br/proposicoesWeb/prop_m...
8,104328,2003-02-18T14:33,2003-05-05T00:00,Devolução ao autor,Arquivada,"Alteração, Regime Jurídico Único, proibição, s...",http://www.camara.gov.br/proposicoesWeb/prop_m...
9,104329,2003-02-18T14:33,2008-03-06T00:00,Notificações,Arquivada,"Criação, Programa de Orientação Sexual, de Pre...",http://www.camara.gov.br/proposicoesWeb/prop_m...


In [73]:
df_proposicoes_detalhadas['statusProposicao_descricaoSituacao'] = df_proposicoes_detalhadas.statusProposicao_descricaoSituacao.fillna('')
df_proposicoes_detalhadas['keywords'] = df_proposicoes_detalhadas.keywords.fillna('')
df_proposicoes_detalhadas['urlInteiroTeor'] = df_proposicoes_detalhadas.urlInteiroTeor.fillna('')

df_proposicoes_detalhadas_unicas = df_proposicoes_detalhadas.drop_duplicates('id', keep='last')

df_proposicoes_detalhadas_unicas

,id,uri,siglaTipo,idTipo,numero,ano,ementa,dataApresentacao,uriOrgaoNumerador,uriUltimoRelator,...,descricaoTipo,ementaDetalhada,keywords,uriPropPrincipal,uriPropAnterior,uriPropPosterior,urlInteiroTeor,urnFinal,texto,justificativa
0,103385,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,7511,2003,"Altera dispositivos da Lei nº 7.501, de 27 de ...",2003-01-03T15:54,NaN,NaN,...,Projeto de Lei,NaN,"Alteração, Regime Jurídico, servidor, Serviço ...",NaN,NaN,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,NaN,NaN
1,103442,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,7512,2003,"Prorroga a vigência da Lei nº 8.989, de 24 de ...",2003-01-02T18:11,NaN,NaN,...,Projeto de Lei,NaN,"Alteração, prorrogação, prazo, vigência, Lei d...",NaN,NaN,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,NaN,NaN
2,103648,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,7514,2003,"Altera a Lei nº 7.070, de 20 de dezembro de 19...",2003-01-14T15:46,NaN,NaN,...,Projeto de Lei,NaN,"Alteração, lei federal, concessão, pensão espe...",NaN,NaN,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,NaN,NaN
3,104314,https://dadosabertos.camara.leg.br/api/v2/prop...,PEC,136,1,2003,Altera a redação do art. 55 da Constituição F...,2003-02-18T14:27,NaN,NaN,...,Proposta de Emenda à Constituição,NaN,"Alteração, Constituição Federal, Legislativo, ...",NaN,NaN,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,NaN,NaN
4,104324,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,2,2003,"Altera o Decreto - Lei nº 2.848, de 7 de dezem...",2003-02-18T14:31,NaN,NaN,...,Projeto de Lei,NaN,"Alteração, legislação trabalhista, (CLT), proi...",https://dadosabertos.camara.leg.br/api/v2/prop...,NaN,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,NaN,NaN
5,104325,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,3,2003,"Altera o art. 129, do Decreto - Lei nº 2.848, ...",2003-02-18T14:31,NaN,NaN,...,Projeto de Lei,NaN,"Alteração, Código Penal, caracterização, crime...",NaN,NaN,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,NaN,NaN
6,104326,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,4,2003,Proíbe a participação de agentes públicos poli...,2003-02-18T14:32,NaN,NaN,...,Projeto de Lei,NaN,"Proibição, participação, policial, Polícia Fed...",NaN,NaN,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,NaN,NaN
7,104327,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,5,2003,"Altera os arts. 1º e 20 da Lei nº 7.716, de 5 ...",2003-02-18T14:32,NaN,NaN,...,Projeto de Lei,NaN,"_ Alteração, Lei Anti-Racismo, penalidade, inf...",NaN,NaN,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,NaN,NaN
8,104328,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,6,2003,"Altera a Lei nº 8.112, de 11 de dezembro de 19...",2003-02-18T14:33,NaN,NaN,...,Projeto de Lei,NaN,"Alteração, Regime Jurídico Único, proibição, s...",NaN,NaN,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,NaN,NaN
9,104329,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,7,2003,Dispõe sobre a criação de Programa de Orientaç...,2003-02-18T14:33,NaN,NaN,...,Projeto de Lei,NaN,"Criação, Programa de Orientação Sexual, de Pre...",NaN,NaN,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,NaN,NaN


In [74]:
dataframe_to_database(
    dataframe=df_proposicoes_detalhadas_unicas,
    tabela='camara_proposicao_detalhada',
    colunas_tabela=[
        'id',
        'data_apresentacao',
        'data_ultimo_status',
        'descricao_tramitacao',
        'descricao_situacao',
        'keywords',
        'url_integra'
    ],
    colunas_dataframe=[
        'id',
        'dataApresentacao',
        'statusProposicao_dataHora',
        'statusProposicao_descricaoTramitacao',
        'statusProposicao_descricaoSituacao',
        'keywords',
        'urlInteiroTeor'
    ]
)

38333 de 38298